In [1]:
import json
import polars as pl
import re

In [2]:
data_dir = '/Volumes/PortableSSD/CSS/data/processed/'

#### Get metaphor matched comments to run through FrameBERT

In [3]:
data_df = pl.read_csv(data_dir+'politics_sample.csv')
with open(data_dir+'exact_matches_politics_42_10000_v2_latest.json') as f:
    exact_matches = json.load(f)
with open(data_dir+'sem_matches_politics_42_10000_v2_latest.json') as f:
    sem_matches = json.load(f)

In [4]:
data_df.head()

id,created_utc,subreddit,category,super_category,body,author
str,i64,str,str,str,str,str
"""er1nbsy""",1560434110,"""AOC""","""politics_2019""","""politics""","""Lol. I was ask…","""Just_WoW_Thing…"
"""gq5rux6""",1615164573,"""AOC""","""politics_2019""","""politics""","""stfu Liberal.""","""gbsedillo20"""
"""gj3jztp""",1610531437,"""AOC""","""politics_2019""","""politics""",""" &gt;Fuck that…","""64590949354397…"
"""esrc8yq""",1562221646,"""AOC""","""politics_2019""","""politics""","""Cauliflower is…","""TobiKato"""
"""gyjpk7u""",1621327024,"""AOC""","""politics_2019""","""politics""","""If they build …","""the_lonely_gam…"


##### Get list of metaphor matches with unigram metaphors

In [5]:
imp_metas = ['game', 'card', 'pass', 'fan' 'pro', 'goal',
             'score', 'draw', 'bush', 'odds', 'course',
            ]

In [6]:
exact_ids = []
for key, matches in exact_matches.items():
    if key in imp_metas:
        if len(matches[1]) > 0:
            exact_ids.extend(matches[1])

imp_list = data_df.filter(pl.col('id').is_in(exact_ids))['body'].to_list()

In [13]:
imp_list[50]

"Of course he's going to hire yes men. He values loyalty to him above knowledge or competence. \n\nOf course there's an enemies list. It's Trump we're talking about. He doesn't like anyone who doesn't praise him. The question is how much action will be taken against those who threaten his ego- and that isn't something anyone knows yet. I think it's highly dependent upon if the people shut up now that he's elected or not. "

##### After running throught FrameBERT

In [3]:
meta_res_df = pl.read_csv(data_dir+'meta_predictions.csv')

In [7]:
meta_res_df.head()

Tokens,Borderline_metaphor,Real_metaphors,Frame_label
str,i64,i64,str
"""Would""",0,0,"""_"""
"""be""",0,0,"""_"""
"""interested""",0,0,"""Experiencer_fo…"
"""to""",0,0,"""_"""
"""know""",0,0,"""Awareness"""


##### Get FrameBERT metaphor matches (true positives)

In [22]:
tokens = meta_res_df['Tokens'].to_list()
metas = meta_res_df['Real_metaphors'].to_list()
frames = meta_res_df['Frame_label'].to_list()

In [31]:
sent = []
meta_words = []
frame_labels = []
data = []

for t in range(len(tokens)):
    if tokens[t] is None:
        continue
    sent.append(tokens[t])
    if metas[t] == 1 and tokens[t] in imp_metas:
        meta_words.append(tokens[t])
        frame_labels.append(frames[t])
    if(tokens[t] =='.'):
        if len(sent) > 0 and len(meta_words) > 0:
            data.append((' '.join(sent), meta_words, frame_labels))
        sent = []
        meta_words = []
        frame_labels = []

In [32]:
data[1]

('Its not like Trump can use the age card when they are not even 5 years apart .',
 ['card'],
 ['_'])

In [33]:
data[0]

("'' Ah , pulling the race card as well .", ['card'], ['_'])

In [34]:
data[3]

('I don ’ t follow the charity game too closely but I sometimes run into people who are big supporters of the Effective Altruistic movement .',
 ['game'],
 ['_'])

In [35]:
data[4]

('If you wanted to play that game and define any fetal death as murder , then God is quite needlessly cruel indeed .',
 ['game'],
 ['_'])

In [36]:
data[5]

('Really not sure what the end game for this Conservative group is , with the kind of money there spending they have to know this will more then likely reduce their influence rather then grow it .',
 ['game'],
 ['_'])

In [37]:
data[6]

("Does that picture look like a guy who gives two fucks about the bill ? Labour refuse an election because they know the country is NOT WITH THEM I believe this is Borises 's plan ' A ' as under these circumstances he will refuse to ask the Queen to sign the Extension Agreement and rightly so ! Giving him 'Game Set Match ! ' We Love Boris ( Prob wo n't even get through the Lords tho ) So inquiring on why mods remove things is ... wrong ? Maybe curious ? All I said was it did make sense , if it didn ’ t to you OK ? ? ? Not sure what your end game is here .",
 ['game'],
 ['_'])

In [38]:
data[7]

("You 're moving the goal post to one of effectiveness at implementing or changing law ( despite Trump doing neither ) .",
 ['goal'],
 ['Purpose'])

In [39]:
data[8]

("I 'm fully expecting the conservatives to play the split the vote on the center and center left card and rely on their base to win .",
 ['card'],
 ['_'])

In [40]:
data[9]

('If I survey a population 20 % black and 80 % white , with a goal of 100 people surveyed , 80 might be enough for a representative sample but 20 might not .',
 ['goal'],
 ['Purpose'])

In [41]:
data[10]

("What you are suggesting is ethnic cleansing : https : //en.wikipedia.org/wiki/Ethnic_cleansing Do you believe that forcefully removing the people of Israel is fair game ? All this talk of liquidity or injecting money back into the economy is missing the literal main point , that it 's deeply inequitable to bail out a segment of the population and burden those costs on other people .",
 ['game'],
 ['_'])

##### Get FrameBERT non metaphor matches (false positives)

In [42]:
sent = []
meta_words = []
frame_labels = []
data = []

for t in range(len(tokens)):
    if tokens[t] is None:
        continue
    sent.append(tokens[t])
    if metas[t] == 0 and tokens[t] in imp_metas:
        meta_words.append(tokens[t])
        frame_labels.append(frames[t])
    if(tokens[t] =='.'):
        if len(sent) > 0 and len(meta_words) > 0:
            data.append((' '.join(sent), meta_words, frame_labels))
        sent = []
        meta_words = []
        frame_labels = []

In [43]:
data[0]

('Would be interested to know if you made this argument pre referendum ? Because if not it certainly seems a lot like changing the rules after the game has finished - or in other words , closing the stable doors after the horse has bolted .',
 ['game'],
 ['_'])

In [44]:
data[1]

('Edit- I was going to keep going through the video and his questions , update my comment as I went , but I dislike listening to Peter Joseph so much I decided to pass .',
 ['pass'],
 ['Traversing'])

In [45]:
data[2]

('Opening ourselves is not how you pass bills .', ['pass'], ['_'])

In [46]:
data[3]

("Maybe working at one of the K-street brothels , he 'll finally get a chance to write and pass some actual fucking legislation .",
 ['pass'],
 ['Sign_agreement'])

In [47]:
data[4]

("When you win a game of chess , do you `` win '' it because it 's chess ? Everyone 's strategy , from day one , is to win the electoral college .",
 ['game'],
 ['_'])

In [48]:
data[5]

('This will pass and truth will eventually set you losers free .',
 ['pass'],
 ['Process_end'])

In [49]:
data[6]

('so the only non arbitrary line you can draw is either conception or death .',
 ['draw'],
 ['Building'])

In [50]:
data[7]

("It 's a completely different game .", ['game'], ['_'])

In [51]:
data[8]

("& gt ; Who cares about the nobility of anyone 's goals , the point is that the interests of the working class and the interest of the capitalist class are at odds .",
 ['odds'],
 ['Competition'])

In [52]:
data[9]

("Your next point will attempt to draw a connection between taxation and those situations , which you ca n't do as they are not equivalent situations .",
 ['draw'],
 ['Building'])

In [53]:
data[10]

('that passes those rights to them when they apply for a green card .',
 ['card'],
 ['_'])

#### Running FrameBERT on non metaphor matched (our algo) samples

In [79]:
data_df = pl.read_csv(data_dir+'politics_sample.csv')
data_df = data_df.sample(fraction=1.0, shuffle=True, seed=42).drop_nulls()

comments = [comment.replace("'", '') for comment in data_df['body'].to_list()]
comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in comments]
ids = data_df['id'].to_list()

comments_long = []
ids_long = []
# filter by char 
for c in range(len(comments)):
    if len(comments[c]) >= 150:
        comments_long.append(comments[c])
        ids_long.append(ids[c])

comments = comments_long[:10000]
ids = ids_long[:10000]

In [80]:
data_df = data_df.filter(pl.col('id').is_in(ids))

exact_ids = []
for key, matches in exact_matches.items():
    if len(matches[1]) > 0:
        exact_ids.extend(matches[1])

non_meta_list = data_df.filter(~pl.col('id').is_in(exact_ids))['body'].to_list()
non_meta_list = non_meta_list[:1000]

##### After running throught FrameBERT

In [112]:
non_meta_res_df = pl.read_csv(data_dir+'non_meta_predictions.csv')
non_meta_res_df.head()

Tokens,Borderline_metaphor,Real_metaphors,Frame_label
str,i64,i64,str
"""Yes""",0,0,"""_"""
""",""",0,0,"""_"""
"""he""",0,0,"""_"""
"""would""",0,0,"""_"""
"""“""",0,0,"""_"""


##### Get FrameBERT metaphor matches

In [113]:
tokens = non_meta_res_df['Tokens'].to_list()
metas = non_meta_res_df['Real_metaphors'].to_list()
#metas = non_meta_res_df['Borderline_metaphor'].to_list()
frames = non_meta_res_df['Frame_label'].to_list()

In [114]:
sent = []
meta_words = []
frame_labels = []
data = []

for t in range(len(tokens)):
    if tokens[t] is None:
        continue
    sent.append(tokens[t])
    if metas[t] == 1:
        meta_words.append(tokens[t])
        frame_labels.append(frames[t])
    if(tokens[t] =='.'):
        if len(sent) > 0 and len(meta_words) > 0:
            data.append((' '.join(sent), meta_words, frame_labels))
        sent = []
        meta_words = []
        frame_labels = []

In [118]:
data[0]

('Yes , he would “ generally ” agree it impeded .', ['impeded'], ['Hindering'])

In [119]:
data[1]

("Why ? Because you say so ? I 'll take the President and CEO of a multi-billion dollar company to handle classified material than a demented witch who has , on numerous occasions , mishandled classified material .",
 ['witch'],
 ['_'])

In [120]:
data[2]

('For the jobs that Lowes is laying off , there is another company taking on those jobs because the people at the store level are not doing them correctly .',
 ['level'],
 ['Rank'])

In [121]:
data[3]

("Does n't affect me personally , I am Canadian watching the US shit-show election from afar ... however the laws around impeachment in the US are very unclear .",
 ['shit-show'],
 ['_'])

In [122]:
data[4]

("Server security is an issue but it 's being used to distract from the context of the emails .",
 ['distract', 'context'],
 ['Protecting', '_'])

In [123]:
data[5]

("It 's easy to spin this to `` DNC hacked by Russia '' , and lose the original focus .",
 ['spin'],
 ['_'])

In [124]:
data[6]

('Another is a foreign government taking action for the purpose of disrupting another country for their benefit .',
 ['disrupting'],
 ['Hindering'])

In [125]:
data[7]

("`` Next year China is going to release a virus way deadlier and it 's going to cripple America because only liberal pansies will be left .",
 ['cripple'],
 ['Cause_harm'])

In [126]:
data[8]

('[ McConnell blocks democracy it every time it comes to him .',
 ['blocks'],
 ['_'])

In [127]:
data[9]

("] ( https : //www.youtube.com/watch ? time_continue=1 & amp ; v=TfQij4aQq1k ) Anytime people start jabbering about `` receipts '' , I only think of Avenatti and how much confidence this sub had on him and `` he definitely has the receipts ! '' daily claim on here .",
 ['claim'],
 ['Statement'])

In [128]:
data[10]

('The deep state is real .', ['deep'], ['_'])